In [65]:
#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')


In [66]:
#import libraries
import pandas as pd 
import numpy as np
#reading the dataset
dataset=pd.read_csv('day.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.columns

In [ ]:
dataset.describe()

In [ ]:
dataset.info()

In [72]:
#assign string values
#1 is spring, 2 is summer, 3 is fall, 4 is winter
dataset.loc[(dataset['season']==1),'season']='spring'
dataset.loc[(dataset['season']==2),'season']='summar'
dataset.loc[(dataset['season']==3),'season']='fall'
dataset.loc[(dataset['season']==4),'season']='winter'

In [ ]:
dataset['season'].astype('category').value_counts()

In [ ]:
#0 is 2018 and 1 is 2019
dataset['yr'].astype('category').value_counts()

In [75]:
# Defining a function to map numeric month values to string month names
def object_map_mnths(x):
    return x.map({
        1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 
        6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sept', 10: 'Oct', 
        11: 'Nov', 12: 'Dec'
    })

# Applying the mapping function to the 'mnth' column
dataset[['mnth']] = dataset[['mnth']].apply(object_map_mnths)


In [ ]:
dataset['mnth'].astype('category').value_counts()

In [ ]:
dataset['holiday'].astype('category').value_counts()

In [78]:
# Defining a function to map numeric weekday values to string weekday names
def str_map_weekday(x):
    return x.map({
        1: 'Mon', 2: 'Tues', 3: 'Wed', 4: 'Thurs', 
        5: 'Fri', 6: 'Sat', 0: 'Sun'
    })


In [79]:

# Applying the mapping function to the 'weekday' column
dataset[['weekday']] = dataset[['weekday']].apply(str_map_weekday)


In [ ]:
dataset[['weekday']].astype('category').value_counts()

In [ ]:
dataset[['workingday']].astype('category').value_counts()

In [82]:
# Assigning string values to different weather conditions
# 1 = Clear, few clouds, partly cloudy
dataset.loc[dataset['weathersit'] == 1, 'weathersit'] = 'A'

# 2 = Mist, Cloudy
dataset.loc[dataset['weathersit'] == 2, 'weathersit'] = 'B'

# 3 = Light Snow, Heavy Rain
dataset.loc[dataset['weathersit'] == 3, 'weathersit'] = 'C'


In [ ]:

# Changing the 'weathersit' column type to category and counting the occurrences of each category
dataset['weathersit'] = dataset['weathersit'].astype('category')
dataset['weathersit'].value_counts()


In [84]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
sns.distplot(dataset['temp'])
plt.show()

In [ ]:
#actual temparature
sns.distplot(dataset['atemp'])
plt.show()

In [ ]:
#wind speed
sns.distplot(dataset['windspeed'])
plt.show()

In [ ]:
#target variable : total count of rental bikes including both casual and registered
sns.distplot(dataset['cnt'])
plt.show()

In [89]:
# Converting the 'dteday' column to datetime format with specific precision
dataset['dteday'] = dataset['dteday'].astype('datetime64[ns]')



In [ ]:

# Selecting only categorical columns, excluding specific data types
dataset_categorical = dataset.select_dtypes(exclude=['float64', 'datetime64', 'int64'])
dataset_categorical.drop('weathersit',axis=1,inplace=True)
# Displaying the categorical columns (though this line is incomplete in the image)
print(dataset_categorical.columns)


In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(3,3,1)
sns.boxplot(x = 'season' , y = 'cnt', data=dataset)
plt.subplot(3,3,2)
sns.boxplot(x = 'mnth' , y = 'cnt', data=dataset)
plt.subplot(3,3,3)
sns.boxplot(x = 'weekday' , y = 'cnt', data=dataset)
plt.subplot(3,3,4)
sns.boxplot(x = 'weathersit' , y = 'cnt', data=dataset)
plt.subplot(3,3,5)
sns.boxplot(x = 'workingday' , y = 'cnt', data=dataset)
plt.subplot(3,3,6)
sns.boxplot(x = 'yr' , y = 'cnt', data=dataset)
plt.subplot(3,3,7)
sns.boxplot(x = 'holiday' , y = 'cnt', data=dataset)
plt.show()



In [ ]:
# List of integer variables to be converted to float
intVarlist = ["casual", "registered", "cnt"]

# Converting the specified columns to float data type
for var in intVarlist:
    dataset[var] = dataset[var].astype("float")

# Selecting only the numeric columns (float64) and displaying the first few rows
dataset_numeric = dataset.select_dtypes(include=['float64'])
dataset_numeric.head()


In [ ]:
sns.pairplot(dataset_numeric)
plt.show()

In [ ]:
cor=dataset_numeric.corr()
cor

In [ ]:
# Creating a mask for the upper triangle of the heatmap
mask = np.array(cor)
mask[np.tril_indices_from(mask)] = False

# Setting up the figure and axis for the plot
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)

# Creating the heatmap with a mask, setting vmax and enabling annotations
sns.heatmap(cor, mask=mask, vmax=1, square=True, annot=True)

# Displaying the plot
plt.show()


In [96]:
dataset.drop('atemp',axis=1,inplace=True)

In [ ]:
dataset.head()

In [98]:
# Selecting only categorical columns (object dtype)
dataset_categorical = dataset.select_dtypes(include=['object'])


In [ ]:
# Displaying the first few rows of the categorical dataset
dataset_categorical.head()


In [ ]:

# Converting categorical variables into dummy/indicator variables
dataset_dummies = pd.get_dummies(dataset_categorical, drop_first=True)

# Ensure that the boolean values are converted to integers (0 and 1)
dataset_dummies = dataset_dummies.astype(int)

# Displaying the first few rows of the dataset with dummy variables
dataset_dummies.head()

In [ ]:
dataset=dataset.drop(list(dataset_categorical.columns),axis=1)
dataset

In [102]:
dataset=pd.concat([dataset,dataset_dummies],axis=1)

In [ ]:
dataset=dataset.drop(['instant','dteday'],axis=1,inplace=False)
dataset.head()

In [ ]:
# Importing necessary libraries for linear regression
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

# Splitting the dataset into train and test sets
from sklearn.model_selection import train_test_split

# Setting a random seed for reproducibility
np.random.seed(0)

# Splitting the dataset: 70% training, 30% testing
df_train, df_test = train_test_split(dataset, train_size=0.7, test_size=0.3, random_state=100)

# Displaying the training set
df_train


In [ ]:
# Importing MinMaxScaler from sklearn
from sklearn.preprocessing import MinMaxScaler

# Initializing the MinMaxScaler
scaler = MinMaxScaler()

# List of columns to apply scaling to (excluding dummy variables)
var = ["temp", "hum", "windspeed", "casual", "registered", "cnt"]

# Applying the scaler to the selected columns in the training set
df_train[var] = scaler.fit_transform(df_train[var])

# Displaying summary statistics of the training data
df_train.describe()


In [ ]:
# Exclude non-numeric columns from df_train before calculating correlation
df_train_numeric = df_train.select_dtypes(include=[np.number])

# Checking the correlation coefficients to see which variables are highly correlated
plt.figure(figsize=(30, 30))
sns.heatmap(df_train_numeric.corr(), annot=True, cmap="YlGnBu")
plt.show()


In [782]:
# Dividing the data into X (features) and y (target)
x_train = df_train.drop(["casual", "registered"], axis=1)
y_train = df_train.pop('cnt')



In [ ]:

# Displaying the first few rows of the X_train dataset
x_train.head()


In [ ]:

# Converting X_train to a NumPy array
np.array(df_train)


In [ ]:
#test
df_train.dtypes


In [786]:
#x_train = pd.get_dummies(x_train, columns=['weathersit'], drop_first=True)


In [ ]:
import statsmodels.api as sm
x_train_lm = sm.add_constant(x_train)
lr = sm.OLS(y_train, x_train_lm).fit()
print(lr.params)


In [ ]:
lm=LinearRegression()
lm.fit(x_train,y_train)

In [ ]:
print(lm.coef_)
print(lm.intercept_)

In [ ]:
lr.summary()

In [ ]:
# Import RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Create a linear regression model
lm = LinearRegression()

# Initialize RFE with 15 features
rfe1 = RFE(lm, 15)

# Fit RFE with the training data
rfe1.fit(x_train, y_train)

# Print the support (True/False for selected features) and ranking
print(rfe1.support_)
print(rfe1.ranking_)

# Get the names of the selected features
col1 = x_train.columns[rfe1.support_]
print(col1)


In [ ]:
# Import variance inflation factor
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Dropping constant column (if present)
a = x_train_rfe1.drop('const', axis=1)

# Evaluating VIFs
vif = pd.DataFrame()
vif['features'] = a.columns
vif['VIF'] = [variance_inflation_factor(a.values, i) for i in range(a.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)

# Sorting the VIF values in descending order
vif = vif.sort_values(by='VIF', ascending=False)

# Displaying the VIF DataFrame
vif


In [ ]:
# Perform RFE with 7 features
lm = LinearRegression()
rfe2 = RFE(lm, 7)

# Fit RFE with training data
rfe2.fit(x_train, y_train)
print(rfe2.support_)
print(rfe2.ranking_)

# Get selected feature columns
col2 = x_train.columns[rfe2.support_]

# Select the chosen features
x_train_rfe2 = x_train[col2]

# Add a constant and fit an OLS model
import statsmodels.api as sm
x_train_rfe2 = sm.add_constant(x_train_rfe2)
lm2 = sm.OLS(y_train, x_train_rfe2).fit()

# Display the summary of the OLS regression results
lm2.summary()


In [ ]:
# Dropping constant column (if present)
b = x_train_rfe2.drop('const', axis=1)

# Calculating VIF for each feature
vif1 = pd.DataFrame()
vif1['features'] = b.columns
vif1['VIF'] = [variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif1['VIF'] = round(vif1['VIF'], 2)

# Sorting VIF values in descending order
vif1 = vif1.sort_values(by='VIF', ascending=False)

# Displaying the VIF DataFrame
vif1


In [ ]:
# Predicting using the OLS model
y_train_cnt = lm2.predict(x_train_rfe2)

# Plotting the distribution of predictions
fig = plt.figure()
sns.distplot((y_train, y_train_cnt), bins=20)
plt.show()


In [ ]:
# Transforming test data using the same scaler
df_test[var] = scaler.transform(df_test[var])
df_test


In [ ]:
# Separating the target variable and dropping unnecessary columns
y_test = df_test.pop('cnt')
x_test = df_test.drop(["casual", "registered"], axis=1)

# Displaying the first few rows of x_test
x_test.head()


In [ ]:
# Dropping the constant from training data
c = x_train_rfe2.drop('const', axis=1)

# Extracting the column names from the training data
col2 = c.columns

# Selecting the same features from the test data
x_test_rfe2 = x_test[col2]

# Adding a constant to the test data
x_test_rfe2 = sm.add_constant(x_test_rfe2)

# Displaying the structure of the test data
x_test_rfe2.info()


In [ ]:
# Predicting on the test data
y_pred = lm2.predict(x_test_rfe2)

# Plotting the predicted vs actual values
plt.figure()
plt.scatter(y_test, y_pred)
plt.show()


In [ ]:
# Importing the R-squared metric and calculating R-squared
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)


In [ ]:
# Plotting a heatmap of the correlation matrix for the selected features
plt.figure(figsize=(8,5))
sns.heatmap(dataset[col2].corr(), cmap="YlGnBu", annot=True)
plt.show()
